This project demonstrates the ability to analyse Windows Event and Sysmon IDs from a server log to detect suspicious activity indicative of potential security incidents.

Developed Python scripts using `pandas`, `numpy`, `csv`, and `datetime` libraries to parse and analyse log data for events such as privilege escalation attempts, explicit credential logons, command-line executions involving network discovery and file transfers, executable file creations, and possible process injections, and produce alerts that were mapped to relevant MITRE ATT&CK techiques to contextualise adversary behaviour and support structured threat detection.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import csv
import re
from datetime import datetime
from itertools import product

In [2]:
# Bring in csv data into a dataframe

data_frame = pd.read_csv('mock_data.csv')
pd.set_option('display.max_colwidth', None)

<ipython-input-2-2fae600caefc>:3: DtypeWarning: Columns (15,16,74) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frame = pd.read_csv('mock_data.csv')




---



# Privilege Escalation detection

In [5]:
def check_possible_privilege_escalation(dataframe):
  privilege_related_wids = [
            4703, 4704, 4732
            ]
  if dataframe['winlog.event_id'].isin(privilege_related_wids).any() or (
    ((dataframe['winlog.event_id'] == 12).any()) and
    (dataframe['winlog.event_data.TargetObject'].astype(str).str.contains(r"\\SAM|\\Control\\Lsa|HKLM\\SECURITY", regex=True, case=False)).any()):
    print(f"\033[1;31mAlert: Possible Privilege Escalation detected \nT1098.001 Account Manipulation: Local Accounts | T1134 Account Token Manipulation\033[0m")
    return True

  #  if dataframe['winlog.event_id'].isin(privilege_related_wids).any() or ((((dataframe['winlog.event_id'] == 12).any()) and (dataframe['winlog.event_data.TargetObject'].astype(str).str.contains(r"\\SAM" or "\\Control\\Lsa" or "HKLM\\SECURITY", regex=True))).any()):
  #    print(f"\033[1;31mAlert: Possible Privilege Escalation detected\033[0m")
  #    return True

def id_4732_details(dataframe):
  # df prep
  dataframe = dataframe[dataframe['winlog.event_id'].isin([4732])].copy()
  dataframe['message'] = dataframe['message'].str.replace("\n", " ", regex=True).replace("\t"," ", regex=True)

  # extract subject account name and account domain, the users who was added to the security group
  snippet = dataframe['message'].str.extract(r'(\A.*\.)') # this outputs the start of the comment until first fullstop - "short snippet"
  account_name = dataframe['message'].str.extract(r'Account Name:  (.*?)  Account Domain')
  dom_account = dataframe['message'].str.extract(r'Account Domain:  (.*?)  Logon ID') # this outputs account domain
  users_added = dataframe['message'].str.extract(r'Member:  Security ID:  (S[-\d]+)') # this outpus the members (SID) who were added to the group
  sec_group_id = dataframe['message'].str.extract(r'Group:  Security ID:  (S[-\d]+)') # this outputs the GSID of the security group
  group_domain = dataframe['message'].str.extract(r'Domain:(.+?)Domain:  (.*?)  Additional Information').iloc[:, 1:2] # this outputs the domain of the security group

  # rename the column names
  snippet.columns=['snippet']
  account_name.columns=['account_name']
  dom_account.columns=['account_domain']
  users_added.columns=['users_added']
  sec_group_id.columns=['sec_group_id']
  group_domain.columns=['group_domain']

  data_member_added_c = pd.concat((dataframe, snippet, account_name, dom_account, users_added, sec_group_id, group_domain), axis=1)

  # collating information
  event_count = data_member_added_c.shape[0]
  id_event = np.unique(data_member_added_c['winlog.event_id'])
  snippet = np.unique(data_member_added_c['snippet'])
  snippet1 = [snip for snip in np.unique(data_member_added_c['snippet'])]
  group_count = np.unique(data_member_added_c['winlog.event_data.TargetUserName']).size

  for i in np.unique(data_member_added_c['winlog.event_data.TargetUserName']):
    GSID = np.unique(data_member_added_c.loc[data_member_added_c['winlog.event_data.TargetUserName'] == i, 'sec_group_id'].values)
    domain = np.unique(data_member_added_c.loc[data_member_added_c['winlog.event_data.TargetUserName'] == i, 'group_domain'].values)

    subject = np.unique(data_member_added_c.loc[data_member_added_c['winlog.event_data.TargetUserName'] == i, 'account_name'].values)
    logon_ID = np.unique(data_member_added_c.loc[data_member_added_c['winlog.event_data.TargetUserName'] == i, 'winlog.event_data.SubjectLogonId'].values)
    subject_ID = np.unique(data_member_added_c.loc[data_member_added_c['winlog.event_data.TargetUserName'] == i, 'winlog.event_data.SubjectUserSid'].values)
    users_count = np.unique(data_member_added_c['users_added']).size

    for j in subject:
      users = data_member_added_c.loc[(data_member_added_c['winlog.event_data.TargetUserName'] == i) & (data_member_added_c['account_name'] == j), 'users_added'].values

    print(f"\n\033[1mEvent ID {id_event}: {str(snippet)[2:-3]}\033[0m - \033[1m{event_count}\033[0m records. A total of \033[1m{group_count}\033[0m group membership(s) was/were changed.")
    print(f"\nGroup \033[1m'{i}'\033[0m, SID: {GSID}, Domain: {str(domain)[1:-1]} was changed.")
    print(f"\nThe group received {users_count} new user(s), added by \033[1m{str(subject)[1:-1]}\033[0m, SID: {subject_ID}, Logon ID: {logon_ID}")
    print(f"\nThe users were:\n {users}")


def privilege_escalation_analyse(dataframe):
  if check_possible_privilege_escalation(dataframe):
    id_4732_details(dataframe)
  else:
    return ("No Privilege Escalation Detected")

## Output:

In [6]:
privilege_escalation_analyse(data_frame)

Alert: Possible Privilege Escalation detected 
T1098.001 Account Manipulation: Local Accounts | T1134 Account Token Manipulation

Event ID [4732]: A member was added to a security-enabled local group - 2 records. A total of 1 group membership(s) was/were changed.

Group 'Remote Desktop Users', SID: ['S-1-5-32-555'], Domain: 'Builtin' was changed.

The group received 2 new user(s), added by 'Administrator', SID: ['S-1-5-21-1230660491-2334538202-2907223834-500'], Logon ID: ['0x1b0617']

The users were:
 ['S-1-5-21-1433274183-2673321112-2382577732-1104'
 'S-1-5-21-1433274183-2673321112-2382577732-1107']




---



# Explicit Credentials Logons

In [7]:
def explicit_cred_logon(dataframe):
  explicit_cred_wids = [
            4648
            ]
  if dataframe['winlog.event_id'].isin(explicit_cred_wids).any():
    print(f"\033[1;31mAlert: Explicit Credential Logon Attempt detected\033[0m")
    return True

def id_4648_details(dataframe):
  # df prep and filter to those accessing critical system, in this case it's the Domain Controller
  filtered_df = data_frame[data_frame['winlog.event_id'].isin([4648])]
  filtered_df_DC = filtered_df['message'].astype(str).str.contains(r"DC\-01", regex=True)
  data_4648 = filtered_df[filtered_df_DC].iloc[:, [0,1,2,5,filtered_df.columns.get_loc('message')]].copy()
  #data_4648 = filtered_df[filtered_df_DC].iloc[:, [0,1,2,5]].copy()
  data_4648['message'] = data_4648['message'].str.replace("\n", "", regex=True).replace("\t","", regex=True)
  data_4648

  # extract information from message
  snippet = data_4648['message'].str.extract(r'(^\A[^.]+\.)')
  process_name = data_4648['message'].str.extract(r'Process Name:([\d|\D|\s|\S]*)Network Information:')
  s_account_name = data_4648['message'].str.extract(r'Account Name:(.*?)Account Domain')
  s_account_domain = data_4648['message'].str.extract(r'Account Domain:(.*?)Logon ID:')
  used_account_name = data_4648['message'].str.extract(r'Account Name:([\d|\D|\s|\S]*)Account Name:([\d|\D|\s|\S]*)Account Domain:').iloc[:, 1:2]
  used_account_domain = data_4648['message'].str.extract(r'Account Domain:([\d|\D|\s|\S]*)Account Domain:([\d|\D|\s|\S]*)Logon GUID:').iloc[:, 1:2]
  target_server = data_4648['message'].str.extract(r'Target Server Name:([\d|\D|\s|\S]*)Additional Information:')
  network_address = data_4648['message'].str.extract(r'Network Address:([\d|\D|\s|\S]*)Port:')
  network_port = data_4648['message'].str.extract(r'Port:([\d|\D|\s|\S]*)This event is generated')
  s_SID = data_4648['message'].str.extract(r'Subject:Security ID:(S[-\d]+)Account Name:')
  s_logon_id = data_4648['message'].str.extract(r'Logon ID:(.*?)Logon GUID:')

  # rename the column names
  snippet.columns=['snippet']
  process_name.columns=['process_name']
  s_account_name.columns=['subject_account_name']
  s_account_domain.columns=['subject_account_domain']
  used_account_name.columns=['used_account_name']
  used_account_domain.columns=['used_account_domain']
  target_server.columns=['target_server']
  network_address.columns=['network_address']
  network_port.columns=['source_port']
  s_SID.columns=['subject_SID']
  s_logon_id.columns=['subject_logon_ID']


  # epoch time understanding
  epoch_list = [datetime.strptime(timestamp, "%b %d, %Y @ %H:%M:%S.%f").timestamp() for timestamp in data_4648['timestamp']]
  data_4648['epoch'] = epoch_list
  epoch_times = data_4648['epoch'].sort_values().tolist()

  # concatenate to create a new df
  logon_expl_cred = pd.concat((data_4648,  snippet, s_SID, s_account_name, s_account_domain, s_logon_id, used_account_name, used_account_domain, target_server, process_name, network_address, network_port), axis=1)
  logon_expl_cred

  # event count
  event_count = logon_expl_cred.shape[0]
  id_event = np.unique(logon_expl_cred['winlog.event_id'])
  snippet = np.unique(logon_expl_cred['snippet'])
  unique_target_server = np.unique(logon_expl_cred['target_server'])
  unique_subject_count = np.unique(logon_expl_cred['subject_account_name']).size
  unique_subject_name = logon_expl_cred['subject_account_name'].values
  unique_subject_name = np.unique(unique_subject_name)
  print(f"\n\033[1mEvent ID {id_event}:\033[0m {str(snippet)[2:-3]} made to \033[1m{unique_target_server}\033[0m - {event_count} records.")

  # time understanding
  threshold_seconds = 10.000 # 10 seconds
  clusters = []
  current_cluster = [epoch_times[0]]

  for i in range(1, len(epoch_times)):
    time_diff = epoch_times[i] - current_cluster[-1]
    if time_diff <= threshold_seconds:
      current_cluster.append(epoch_times[i])
    else:
      clusters.append(current_cluster)
      current_cluster = [epoch_times[i]]

  clusters.append(current_cluster)

  for i, cluster in enumerate(clusters, start=1):
    print(f'Cluster {i}: {len(cluster)} record(s) within {threshold_seconds} seconds starting at {datetime.fromtimestamp(cluster[0]).strftime("%H:%M:%S.%f")[:-3]}')

# body of details
  unique_record = np.unique(logon_expl_cred['winlog.record_id'])
  for i in unique_record:
    timestamp = np.unique(logon_expl_cred.loc[logon_expl_cred['winlog.record_id'] == i, 'timestamp'].values)
    for j in timestamp:
      process_name = np.unique(logon_expl_cred.loc[(logon_expl_cred['winlog.record_id'] == i) & (logon_expl_cred['timestamp'] == j), 'process_name'].values)
      for p in process_name:
        cred_used_account = np.unique(logon_expl_cred.loc[(logon_expl_cred['winlog.record_id'] == i) & (logon_expl_cred['timestamp'] == j) & (logon_expl_cred['process_name'] == p), 'used_account_name'].values)
        for k in cred_used_account:
          subject_name = np.unique(logon_expl_cred.loc[(logon_expl_cred['winlog.record_id'] == i) & (logon_expl_cred['timestamp'] == j) & (logon_expl_cred['process_name'] == p) & (logon_expl_cred['used_account_name'] == k), 'subject_account_name'].values)
          for s in subject_name:
            network_address = np.unique(logon_expl_cred.loc[(logon_expl_cred['winlog.record_id'] == i) & (logon_expl_cred['timestamp'] == j) & (logon_expl_cred['process_name'] == p) & (logon_expl_cred['used_account_name'] == k) & (logon_expl_cred['subject_account_name'] == s), 'network_address'].values)
            for n in network_address:
              network_port = np.unique(logon_expl_cred.loc[(logon_expl_cred['winlog.record_id'] == i) & (logon_expl_cred['timestamp'] == j) & (logon_expl_cred['process_name'] == p) & (logon_expl_cred['used_account_name'] == k) & (logon_expl_cred['subject_account_name'] == s) & (logon_expl_cred['network_address'] == n), 'source_port'].values)

            print(f"\n\tRecord ID: {i}, Time: {str(j)[-12:]}, \n\t\tProcess Name: {p}, \n\t\tCredential Used: {k}, \n\t\tSubject Account: {s}, \n\t\tNetwork Address: {n}, \tNetwork Port: {network_port}")


def explicit_credential_logon_analyse(dataframe):
  if explicit_cred_logon(dataframe):
    id_4648_details(dataframe)
  else:
    print("No Event ID 4648 Detected")

## Output:

In [8]:
explicit_credential_logon_analyse(data_frame)

Alert: Explicit Credential Logon Attempt detected

Event ID [4648]: A logon was attempted using explicit credentials made to ['GMR-DC-01.glenmoret.corp'] - 5 records.
Cluster 1: 3 record(s) within 10.0 seconds starting at 11:40:49.926
Cluster 2: 1 record(s) within 10.0 seconds starting at 13:22:24.659
Cluster 3: 1 record(s) within 10.0 seconds starting at 13:23:50.018

	Record ID: 3908, Time: 11:40:49.926, 
		Process Name: C:\Windows\System32\SystemSettingsAdminFlows.exe, 
		Credential Used: administrator, 
		Subject Account: Administrator, 
		Network Address: -, 	Network Port: ['-']

	Record ID: 3909, Time: 11:40:50.122, 
		Process Name: C:\Windows\System32\SystemSettingsAdminFlows.exe, 
		Credential Used: administrator, 
		Subject Account: Administrator, 
		Network Address: 192.168.100.13, 	Network Port: ['49668']

	Record ID: 3910, Time: 11:40:58.562, 
		Process Name: C:\Windows\System32\SystemSettingsAdminFlows.exe, 
		Credential Used: administrator, 
		Subject Account: Administrat



---



# Check Command Line Execution

In [9]:
def check_command_line(dataframe):
  if (~((data_frame['winlog.event_data.CommandLine'].astype(str) == "-") | (data_frame['winlog.event_data.CommandLine'].astype(str) == ""))).any():
    print(f"\033[1;31mAlert: Command Line execution detected\033[0m")
    return True

def cmd_details(dataframe):
  # prep
  data_cmd = dataframe[~((dataframe['winlog.event_data.CommandLine'].astype(str) == "-") | (dataframe['winlog.event_data.CommandLine'].astype(str) == ""))]
  found_file_transfer = False
  found_discovery = False
  discovery_commands = [ "net1 user", "net user", "net groups" , "ipconfig", "ifconfig.me", "arp", "route", "nslookup", "nbtstat", "tasklist", "systeminfo", "whoami", "ssh", "dir", "tree"]
  file_transfer_commands = ["curl", "ftp", "wget", "scp"]

  # create patterns into a single regex
  file_transfer_pattern = '|'.join(map(re.escape, file_transfer_commands))
  discovery_pattern = '|'.join(map(re.escape, discovery_commands))


  for i in file_transfer_commands:
      if data_cmd['winlog.event_data.CommandLine'].astype(str).str.contains(i, regex=True).any():
          found_file_transfer = True
          break
  for j in discovery_commands:
      if data_cmd['winlog.event_data.CommandLine'].astype(str).str.contains(j, regex=True).any():
          found_discovery = True
          break

  if found_file_transfer and found_discovery:
    print(f"\033[31mPossible file transfer and network/ system discovery command detected: T1105 Ingress Tool Transfer | T1087 Account Discovery | T1049 System Network Connections Discovery\033[0m")

  elif found_file_transfer and not found_discovery:
    print(f"\033[31mPossible file transfer command detected: T1105 Ingress Tool Transfer\033[0m")

  elif not found_file_transfer and found_discovery:
    print(f"\033[31mPossible network or system discovery command detected: T1087 Account Discovery | T1049 System Network Connections Discovery\033[0m")

  else:
    print(f"Purposes to be investigate")


  # For file transfer commands found, create a new dataframe
  if found_file_transfer:
    mask = data_cmd['winlog.event_data.CommandLine'].astype(str).str.contains(file_transfer_pattern, regex=True)
    filtered_file_transfer_df = data_cmd[mask][[
        'timestamp',
        'winlog.record_id',
        'winlog.event_data.User',
        'winlog.event_data.ParentImage',
        'winlog.event_data.Image'
        #'winlog.event_data.CommandLine'
        ]]

    print("\n\033[1;34mFile Transfer command records:\033[0m")

    unique_record = np.unique(filtered_file_transfer_df['winlog.record_id'])
    for i in unique_record:
      timestamp = np.unique(filtered_file_transfer_df.loc[filtered_file_transfer_df['winlog.record_id'] == i, 'timestamp'].values)
      for j in timestamp:
        users = np.unique(filtered_file_transfer_df.loc[(filtered_file_transfer_df['winlog.record_id'] == i) & (filtered_file_transfer_df['timestamp'] == j), 'winlog.event_data.User'].values)
        for u in users:
          parent_image = np.unique(filtered_file_transfer_df.loc[(filtered_file_transfer_df['winlog.record_id'] == i) & (filtered_file_transfer_df['timestamp'] == j) & (filtered_file_transfer_df['winlog.event_data.User'] == u), 'winlog.event_data.ParentImage'].values)
          for p in parent_image:
            images = np.unique(filtered_file_transfer_df.loc[(filtered_file_transfer_df['winlog.record_id'] == i) & (filtered_file_transfer_df['timestamp'] == j) & (filtered_file_transfer_df['winlog.event_data.User'] == u) & (filtered_file_transfer_df['winlog.event_data.ParentImage'] == p), 'winlog.event_data.Image'].values)
          print(f"Record ID: {i}, at {str(j)[-12:]},\n\tUser: {u}, Parent Image: {p}, Image: {images}")


 # For discovery commands found, create a new dataframe
  if found_discovery:
    mask = data_cmd['winlog.event_data.CommandLine'].astype(str).str.contains(discovery_pattern, regex=True)
    filtered_discovery_df = data_cmd[mask][[
        'timestamp',
        'winlog.record_id',
        'winlog.event_data.User',
        'winlog.event_data.ParentImage',
        'winlog.event_data.Image'
        #'winlog.event_data.CommandLine'
        ]]

    print("\n\033[1;34m\nNetwork/ System Discovery command records:\033[0m")


    unique_record = np.unique(filtered_discovery_df['winlog.record_id'])
    for i in unique_record:
      timestamp = np.unique(filtered_discovery_df.loc[filtered_discovery_df['winlog.record_id'] == i, 'timestamp'].values)
      for j in timestamp:
        users = np.unique(filtered_discovery_df.loc[(filtered_discovery_df['winlog.record_id'] == i) & (filtered_discovery_df['timestamp'] == j), 'winlog.event_data.User'].values)
        for u in users:
          parent_image = np.unique(filtered_discovery_df.loc[(filtered_discovery_df['winlog.record_id'] == i) & (filtered_discovery_df['timestamp'] == j) & (filtered_discovery_df['winlog.event_data.User'] == u), 'winlog.event_data.ParentImage'].values)
          for p in parent_image:
            images = np.unique(filtered_discovery_df.loc[(filtered_discovery_df['winlog.record_id'] == i) & (filtered_discovery_df['timestamp'] == j) & (filtered_discovery_df['winlog.event_data.User'] == u) & (filtered_discovery_df['winlog.event_data.ParentImage'] == p), 'winlog.event_data.Image'].values)
          print(f"Record ID: {i}, at {str(j)[-12:]},\n\tUser: {u}, Parent Image: {p}, Image: {images}")


def check_cmd_execution(dataframe):
  if check_command_line(dataframe):
    cmd_details(dataframe)
  else:
    print("No Command Line Detected")

## Output:

In [10]:
check_cmd_execution(data_frame)

Alert: Command Line execution detected
Possible file transfer and network/ system discovery command detected: T1105 Ingress Tool Transfer | T1087 Account Discovery | T1049 System Network Connections Discovery

File Transfer command records:
Record ID: 951744, at 13:15:42.345,
	User: GLENMORET\simon.cooper1, Parent Image: C:\Windows\System32\cmd.exe, Image: ['C:\\Windows\\System32\\curl.exe']
Record ID: 952086, at 13:16:15.952,
	User: GLENMORET\simon.cooper1, Parent Image: C:\Windows\System32\cmd.exe, Image: ['C:\\Windows\\System32\\curl.exe']
Record ID: 952713, at 13:16:58.091,
	User: GLENMORET\simon.cooper1, Parent Image: C:\Windows\System32\cmd.exe, Image: ['C:\\Windows\\System32\\curl.exe']
Record ID: 952776, at 13:17:12.000,
	User: GLENMORET\simon.cooper1, Parent Image: C:\Windows\System32\cmd.exe, Image: ['C:\\Windows\\System32\\curl.exe']


Network/ System Discovery command records:
Record ID: 951696, at 13:15:29.538,
	User: GLENMORET\simon.cooper1, Parent Image: C:\Windows\Syste



---



# Executable File Creation


In [11]:
def check_FileExecutableDetected(dataframe):
  swids = [
            29
            ]
  if dataframe['winlog.event_id'].isin(swids).any() & dataframe['winlog.event_data.TargetFilename'].astype(str).str.contains(r"C:\\Users", regex=True).any():
    print(f"\033[1;31mAlert: Sysmon ID 29 File Executable Creation in C:\\Users Detected \n T1204.002 User Execution: Malicious File\033[0m")
    return True

def id_29_details(dataframe):
  # prep
  data_29 = dataframe.loc[
      (dataframe['winlog.event_id'] == 29) &
      (dataframe['winlog.event_data.TargetFilename'].astype(str).str.contains(r"^C:\\Users", case=False))
  ]

  unique_user = np.unique(data_29['winlog.event_data.User'])
  unique_image = np.unique(data_29['winlog.event_data.Image'])

  combinations = product(unique_user, unique_image)

  results = []
  for user, image in combinations:
      mask = (data_29['winlog.event_data.User'] == user) & (data_29['winlog.event_data.Image'] == image)
      if mask.any():
        filtered = data_29[mask].sort_values('timestamp')
        record_count = filtered.shape[0]
        first_row = filtered.iloc[0]
        results.append((user, image, first_row['winlog.record_id'], first_row['timestamp'], record_count))
  result_df = pd.DataFrame(results, columns=['User', 'Image', 'Record_ID', 'Timestamp', 'Record_Count'])

  for u in np.unique(result_df['User']):
    print(f"\n\033[1mUser: {u}\033[0m:")
    for m in np.unique(result_df.loc[result_df['User'] == u, 'Image'].values):
      for c in np.unique(result_df.loc[(result_df['User'] == u) & (result_df['Image'] == m), 'Record_Count'].values):
        print(f"\tPrcoess: {m} - {c} file(s) created")
        for t in np.unique(result_df.loc[(result_df['User'] == u) & (result_df['Image'] == m)  & (result_df['Record_Count'] == c), 'Timestamp'].values):
          for r in np.unique(result_df.loc[(result_df['User'] == u) & (result_df['Image'] == m) & (result_df['Record_Count'] == c) & (result_df['Timestamp'] == t), 'Record_ID'].values):
            print(f"\t\tFirst file created at {str(t)[-12:]}, Record ID: {r}")

def executablefilecreation(dataframe):
  if check_FileExecutableDetected(dataframe):
    id_29_details(dataframe)
  else:
    print("No Sysmon ID 29 File Executable Creation Detected")

## Output:

In [12]:
executablefilecreation(data_frame)

Alert: Sysmon ID 29 File Executable Creation in C:\Users Detected 
 T1204.002 User Execution: Malicious File

User: GLENMORET\simon.cooper1:
	Prcoess: C:\Users\simon.cooper1.GLENMORET\Desktop\AngryIPScannerPortable\AngryIPScannerPortable.exe - 2 file(s) created
		First file created at 13:18:16.726, Record ID: 953857
	Prcoess: C:\Users\simon.cooper1.GLENMORET\Desktop\AngryIPScannerPortable\App\AngryIPScanner\jre\bin\javaw.exe - 2 file(s) created
		First file created at 13:18:18.063, Record ID: 954045
	Prcoess: C:\Users\simon.cooper1.GLENMORET\Desktop\scanner.paf.exe - 20 file(s) created
		First file created at 13:17:36.292, Record ID: 953220
	Prcoess: C:\Windows\system32\curl.exe - 1 file(s) created
		First file created at 13:17:16.506, Record ID: 952823




---



# Possible Process Injection Detection

In [13]:
def check_CreateRemoteThread(dataframe):
  swids = [
            8
            ]
  if dataframe['winlog.event_id'].isin(swids).any():
    print(f"\033[1;31mTo Investigate: Possible Process Injection detected \nT1055 Process Injection\n\033[0m")
    return True
  else:
    print("No Sysmon ID 8 CreateRemoteThread Detected")


def id_8_details(dataframe):
  # prep
  data_8 = dataframe.loc[(dataframe['winlog.event_id'] == 8)]

  record_count = data_8.shape[0]
  unique_source_process = np.unique(data_8['winlog.event_data.SourceImage'])
  unique_target_process = np.unique(data_8['winlog.event_data.TargetImage'])
  unique_user = np.unique(data_8['winlog.event_data.SourceUser'])

  for u in unique_user:
    print(f"\033[1mUser: {u}\033[0m, Record Count: {record_count}")
    source_process = np.unique(data_8.loc[data_8['winlog.event_data.SourceUser'] == u, 'winlog.event_data.SourceImage'].values)
    for s in source_process:
      target_process = np.unique(data_8.loc[(data_8['winlog.event_data.SourceUser'] == u) & (data_8['winlog.event_data.SourceImage'] == s), 'winlog.event_data.TargetImage'].values)
      for p in target_process:
        start_address = np.unique(data_8.loc[(data_8['winlog.event_data.SourceUser'] == u) & (data_8['winlog.event_data.SourceImage'] == s) &
         (data_8['winlog.event_data.TargetImage'] == p), 'winlog.event_data.StartAddress'].values)
        for a in start_address:
          start_function = np.unique(data_8.loc[(data_8['winlog.event_data.SourceUser'] == u) & (data_8['winlog.event_data.SourceImage'] == s) &
           (data_8['winlog.event_data.TargetImage'] == p) & (data_8['winlog.event_data.StartAddress'] == a), 'winlog.event_data.StartFunction'].values)
          for f in start_function:
            start_module = np.unique(data_8.loc[(data_8['winlog.event_data.SourceUser'] == u) & (data_8['winlog.event_data.SourceImage'] == s) & (data_8['winlog.event_data.TargetImage'] == p) & (data_8['winlog.event_data.StartAddress'] == a) & (data_8['winlog.event_data.StartFunction'] == f), 'winlog.event_data.StartModule'].values)
            for m in start_module:
              print(f"Source Process: {s}, Target Process: {p}, Start Address: {a}, Start Function: {f}, Start Module: {m}")


  combinations = product(unique_user, unique_source_process, unique_target_process)
  results = []

  for user, source_process, target_process in combinations:
      mask = (data_8['winlog.event_data.SourceUser'] == user) & (data_8['winlog.event_data.SourceImage'] == source_process) & (data_8['winlog.event_data.TargetImage'] == target_process)
      if mask.any():
        filtered = data_8[mask].sort_values('timestamp')
        for _, row in filtered.iterrows():
          results.append((row['winlog.record_id'], row['timestamp']))
  result_df = pd.DataFrame(results, columns=['Record_ID', 'Timestamp'])

  print(f"Record ID(s):{', '.join(map(str, result_df['Record_ID'].tolist()))}")

def CreateRemoteThread_detection(dataframe):
  if check_CreateRemoteThread(dataframe):
    id_8_details(dataframe)
  else:
    print("No Sysmon ID 8 CreateRemoteThread Detected")

## Output:

In [14]:
CreateRemoteThread_detection(data_frame)

To Investigate: Possible Process Injection detected 
T1055 Process Injection

User: GLENMORET\tony.barry1, Record Count: 4
Source Process: C:\Windows\System32\rdpclip.exe, Target Process: C:\Windows\System32\csrss.exe, Start Address: 0xFFFF9A65900F2E00, Start Function: -, Start Module: -
Record ID(s):915353, 915372, 921722, 921728




---

